[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/Digital-Finance-Introduction/blob/main/day_02/notebooks/NB03_Open_Banking_API.ipynb)

# NB03: Open Banking API Explorer

**Topic:** 2.2 - Open Banking and Platform Finance

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Open Banking Concepts**: Explain PSD2/PSD3 regulations and their impact on financial services
2. **Simulate API Interactions**: Work with mock banking APIs for account information and transactions
3. **Build a Mock Banking API**: Create a simulated bank server that exposes Open Banking endpoints
4. **Understand OAuth 2.0 Flows**: Grasp authentication concepts used in Open Banking
5. **Aggregate Multiple Banks**: Build a third-party provider that combines data from multiple sources
6. **Appreciate Security Considerations**: Understand the security model of Open Banking

## Section 1: Setup

We'll use Python's standard libraries to simulate Open Banking API interactions. No external API calls are made - everything is simulated locally for educational purposes.

In [ ]:
# Import required libraries
import json
import uuid
import hashlib
import base64
import secrets
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field, asdict
from enum import Enum
import random

# Set random seed for reproducibility
random.seed(42)

print("Libraries loaded successfully!")
print(f"Python datetime: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nThis notebook simulates Open Banking APIs without real network calls.")
print("All data is mock data for educational purposes.")

## Section 2: What is Open Banking?

### The Revolution in Financial Services

**Open Banking** is a regulatory and technological framework that allows third-party financial service providers to access consumer banking data through secure APIs, with the consumer's consent.

### Key Regulations

| Regulation | Region | Year | Key Features |
|------------|--------|------|-------------|
| **PSD2** | EU | 2018 | Payment Services Directive 2 - mandates banks to open APIs |
| **PSD3** | EU | 2024+ | Enhanced security, instant payments, fraud protection |
| **Open Banking UK** | UK | 2018 | Nine largest banks required to share data |
| **CFPB 1033** | USA | 2024+ | Consumer Financial Protection Bureau data access rule |

### The Three Pillars of Open Banking

```
+-------------------+     +-------------------+     +-------------------+
|                   |     |                   |     |                   |
|   AISP            |     |   PISP            |     |   CISP            |
|   Account         |     |   Payment         |     |   Card-Based      |
|   Information     |     |   Initiation      |     |   Payment         |
|   Service         |     |   Service         |     |   Instrument      |
|   Provider        |     |   Provider        |     |   Issuer          |
|                   |     |                   |     |                   |
+-------------------+     +-------------------+     +-------------------+
        |                         |                         |
        |    READ account data    |    INITIATE payments    |    ISSUE cards
        v                         v                         v
+---------------------------------------------------------------+
|                                                               |
|                    BANK'S OPEN BANKING API                    |
|                                                               |
+---------------------------------------------------------------+
```

### Benefits of Open Banking

**For Consumers:**
- Aggregated view of all accounts in one app
- Better financial products through data sharing
- Easier switching between providers
- Innovative services (budgeting, lending)

**For Fintechs:**
- Access to banking data without screen scraping
- Ability to initiate payments directly
- Level playing field with traditional banks
- New business models possible

**For Banks:**
- New revenue streams through APIs
- Partnership opportunities
- Innovation pressure driving improvement
- Customer data insights

In [ ]:
# Visualize the Open Banking Ecosystem

def display_open_banking_ecosystem():
    """
    Display the Open Banking ecosystem participants and their relationships.
    """
    print("="*80)
    print("OPEN BANKING ECOSYSTEM".center(80))
    print("="*80)
    
    ecosystem = {
        "Account Holder (PSU)": {
            "role": "Payment Service User",
            "description": "The customer who owns the bank account",
            "actions": ["Grants consent", "Uses services", "Controls data access"]
        },
        "ASPSP (Banks)": {
            "role": "Account Servicing Payment Service Provider",
            "description": "Traditional banks holding customer accounts",
            "actions": ["Provides APIs", "Holds accounts", "Processes payments"]
        },
        "AISP (Aggregators)": {
            "role": "Account Information Service Provider",
            "description": "Third parties that aggregate account information",
            "actions": ["Reads account data", "Provides insights", "Aggregates views"]
        },
        "PISP (Payment Initiators)": {
            "role": "Payment Initiation Service Provider",
            "description": "Third parties that can initiate payments",
            "actions": ["Initiates payments", "Confirms transfers", "Reduces card fees"]
        }
    }
    
    for participant, details in ecosystem.items():
        print(f"\n{participant}")
        print("-" * len(participant))
        print(f"  Role: {details['role']}")
        print(f"  Description: {details['description']}")
        print(f"  Key Actions:")
        for action in details['actions']:
            print(f"    - {action}")
    
    print("\n" + "="*80)
    print("\nData Flow in Open Banking:")
    print("-" * 40)
    print("""    
    1. Customer wants to use a budgeting app (AISP)
    2. App redirects customer to bank for authentication
    3. Customer logs in and grants consent
    4. Bank issues access token to the app
    5. App uses token to access account data via API
    6. Customer sees aggregated data in the app
    """)
    print("="*80)

display_open_banking_ecosystem()

## Section 3: Simulate Account Information API

The **Account Information Service** (AIS) is the most common Open Banking use case. It allows third parties to:
- View account balances
- Access transaction history
- Retrieve account details

Let's build a mock bank with accounts and simulate the API responses.

In [ ]:
# Define data structures for our mock bank

class AccountType(Enum):
    CURRENT = "CurrentAccount"
    SAVINGS = "SavingsAccount"
    CREDIT_CARD = "CreditCard"
    LOAN = "LoanAccount"

class AccountStatus(Enum):
    ENABLED = "Enabled"
    DISABLED = "Disabled"
    BLOCKED = "Blocked"

@dataclass
class Balance:
    """Represents an account balance."""
    amount: float
    currency: str = "EUR"
    credit_line_included: bool = False
    balance_type: str = "ClosingAvailable"
    reference_date: str = field(default_factory=lambda: datetime.now().strftime('%Y-%m-%d'))

@dataclass
class Account:
    """Represents a bank account in Open Banking format."""
    account_id: str
    iban: str
    currency: str
    account_type: AccountType
    status: AccountStatus
    name: str
    balances: List[Balance] = field(default_factory=list)
    owner_name: str = ""
    bic: str = ""
    
    def to_api_response(self) -> Dict:
        """Convert to Open Banking API response format."""
        return {
            "accountId": self.account_id,
            "iban": self.iban,
            "currency": self.currency,
            "accountType": self.account_type.value,
            "status": self.status.value,
            "name": self.name,
            "ownerName": self.owner_name,
            "bic": self.bic,
            "_links": {
                "balances": f"/accounts/{self.account_id}/balances",
                "transactions": f"/accounts/{self.account_id}/transactions"
            }
        }

print("Data structures defined successfully!")
print("\nAccount Types available:")
for at in AccountType:
    print(f"  - {at.value}")

In [ ]:
# Create a mock bank with customer accounts

class MockBank:
    """
    Simulates a bank's Open Banking API.
    This is a simplified version for educational purposes.
    """
    
    def __init__(self, bank_name: str, bic: str):
        self.bank_name = bank_name
        self.bic = bic
        self.customers: Dict[str, Dict] = {}
        self.accounts: Dict[str, Account] = {}
        self.transactions: Dict[str, List[Dict]] = {}
        self.consents: Dict[str, Dict] = {}
        
    def add_customer(self, customer_id: str, name: str, accounts: List[Account]):
        """Add a customer with their accounts."""
        self.customers[customer_id] = {
            "name": name,
            "account_ids": [acc.account_id for acc in accounts]
        }
        for account in accounts:
            account.owner_name = name
            account.bic = self.bic
            self.accounts[account.account_id] = account
            self.transactions[account.account_id] = []
    
    def generate_iban(self, country: str = "DE") -> str:
        """Generate a mock IBAN."""
        bank_code = "".join([str(random.randint(0, 9)) for _ in range(8)])
        account_num = "".join([str(random.randint(0, 9)) for _ in range(10)])
        check_digits = str(random.randint(10, 99))
        return f"{country}{check_digits}{bank_code}{account_num}"
    
    def get_accounts(self, customer_id: str, consent_id: str) -> Dict:
        """API: Get list of accounts (with consent check)."""
        # Verify consent
        if consent_id not in self.consents:
            return {"error": "CONSENT_INVALID", "message": "Consent not found"}
        
        consent = self.consents[consent_id]
        if consent["status"] != "valid":
            return {"error": "CONSENT_EXPIRED", "message": "Consent has expired"}
        
        if customer_id not in self.customers:
            return {"error": "CUSTOMER_NOT_FOUND", "message": "Customer not found"}
        
        customer = self.customers[customer_id]
        accounts_list = []
        
        for acc_id in customer["account_ids"]:
            if acc_id in self.accounts:
                accounts_list.append(self.accounts[acc_id].to_api_response())
        
        return {
            "accounts": accounts_list,
            "_links": {
                "self": "/accounts"
            }
        }
    
    def get_balances(self, account_id: str, consent_id: str) -> Dict:
        """API: Get account balances."""
        if consent_id not in self.consents or self.consents[consent_id]["status"] != "valid":
            return {"error": "CONSENT_INVALID"}
        
        if account_id not in self.accounts:
            return {"error": "ACCOUNT_NOT_FOUND"}
        
        account = self.accounts[account_id]
        return {
            "account": {
                "iban": account.iban
            },
            "balances": [
                {
                    "balanceAmount": {
                        "currency": bal.currency,
                        "amount": str(bal.amount)
                    },
                    "balanceType": bal.balance_type,
                    "creditLimitIncluded": bal.credit_line_included,
                    "referenceDate": bal.reference_date
                } for bal in account.balances
            ]
        }

# Create our mock bank
demo_bank = MockBank("Demo Digital Bank", "DEMODEBKXXX")

print(f"Created mock bank: {demo_bank.bank_name}")
print(f"BIC: {demo_bank.bic}")

In [ ]:
# Populate the bank with sample customers and accounts

def create_sample_accounts(bank: MockBank, num_customers: int = 3):
    """
    Create sample customers with various account types.
    """
    customer_names = [
        "Alice Mueller",
        "Bob Schmidt",
        "Carol Weber",
        "David Fischer",
        "Eva Braun"
    ]
    
    for i in range(min(num_customers, len(customer_names))):
        customer_id = f"CUST{i+1:04d}"
        name = customer_names[i]
        
        accounts = []
        
        # Current account (everyone has one)
        current_balance = round(random.uniform(500, 15000), 2)
        current_acc = Account(
            account_id=f"ACC{i*3+1:06d}",
            iban=bank.generate_iban("DE"),
            currency="EUR",
            account_type=AccountType.CURRENT,
            status=AccountStatus.ENABLED,
            name="Main Current Account",
            balances=[
                Balance(amount=current_balance, balance_type="ClosingAvailable"),
                Balance(amount=current_balance + 200, balance_type="ClosingBooked")
            ]
        )
        accounts.append(current_acc)
        
        # Savings account (70% of customers)
        if random.random() < 0.7:
            savings_balance = round(random.uniform(1000, 50000), 2)
            savings_acc = Account(
                account_id=f"ACC{i*3+2:06d}",
                iban=bank.generate_iban("DE"),
                currency="EUR",
                account_type=AccountType.SAVINGS,
                status=AccountStatus.ENABLED,
                name="Savings Account",
                balances=[Balance(amount=savings_balance)]
            )
            accounts.append(savings_acc)
        
        # Credit card (50% of customers)
        if random.random() < 0.5:
            credit_used = round(random.uniform(0, 3000), 2)
            credit_acc = Account(
                account_id=f"ACC{i*3+3:06d}",
                iban=bank.generate_iban("DE"),
                currency="EUR",
                account_type=AccountType.CREDIT_CARD,
                status=AccountStatus.ENABLED,
                name="Visa Credit Card",
                balances=[
                    Balance(amount=-credit_used, balance_type="ClosingBooked"),
                    Balance(amount=5000-credit_used, balance_type="ClosingAvailable", credit_line_included=True)
                ]
            )
            accounts.append(credit_acc)
        
        bank.add_customer(customer_id, name, accounts)
        
    return bank

# Populate the bank
demo_bank = create_sample_accounts(demo_bank, num_customers=5)

print("Sample customers created:")
print("=" * 60)
for cust_id, cust_data in demo_bank.customers.items():
    print(f"\nCustomer: {cust_data['name']} ({cust_id})")
    print(f"  Accounts: {len(cust_data['account_ids'])}")
    for acc_id in cust_data['account_ids']:
        acc = demo_bank.accounts[acc_id]
        balance = acc.balances[0].amount if acc.balances else 0
        print(f"    - {acc.name}: {balance:,.2f} {acc.currency} ({acc.account_type.value})")

In [ ]:
# Demonstrate the Account Information API

def demo_accounts_api(bank: MockBank, customer_id: str):
    """
    Demonstrate the accounts API endpoint.
    """
    print("\n" + "="*80)
    print("OPEN BANKING API: GET /accounts")
    print("="*80)
    
    # First, we need consent (we'll cover this properly in Section 6)
    consent_id = f"consent-{uuid.uuid4().hex[:8]}"
    bank.consents[consent_id] = {
        "status": "valid",
        "customer_id": customer_id,
        "created": datetime.now().isoformat(),
        "expires": (datetime.now() + timedelta(days=90)).isoformat()
    }
    
    print(f"\nRequest:")
    print(f"  GET /accounts")
    print(f"  Headers:")
    print(f"    Authorization: Bearer <access_token>")
    print(f"    Consent-ID: {consent_id}")
    print(f"    X-Request-ID: {uuid.uuid4()}")
    
    # Make the API call
    response = bank.get_accounts(customer_id, consent_id)
    
    print(f"\nResponse (Status: 200 OK):")
    print(json.dumps(response, indent=2))
    
    return consent_id

# Demo with first customer
consent_id = demo_accounts_api(demo_bank, "CUST0001")

In [ ]:
# Demonstrate the Balances API

def demo_balances_api(bank: MockBank, account_id: str, consent_id: str):
    """
    Demonstrate the balances API endpoint.
    """
    print("\n" + "="*80)
    print(f"OPEN BANKING API: GET /accounts/{account_id}/balances")
    print("="*80)
    
    print(f"\nRequest:")
    print(f"  GET /accounts/{account_id}/balances")
    print(f"  Headers:")
    print(f"    Authorization: Bearer <access_token>")
    print(f"    Consent-ID: {consent_id}")
    
    response = bank.get_balances(account_id, consent_id)
    
    print(f"\nResponse (Status: 200 OK):")
    print(json.dumps(response, indent=2))
    
    # Explain balance types
    print("\nBalance Types Explained:")
    print("-" * 40)
    print("  ClosingAvailable: Funds available right now for transactions")
    print("  ClosingBooked: Total of all posted transactions")
    print("  Expected: Including pending transactions")
    print("  InterimAvailable: Intraday available balance")

# Demo with first account
demo_balances_api(demo_bank, "ACC000001", consent_id)

## Section 4: Simulate Transaction History API

The **Transactions API** provides access to account transaction history. This is crucial for:
- Personal finance management apps
- Credit scoring services
- Accounting software integration
- Fraud detection systems

In [ ]:
# Transaction data structures

class TransactionStatus(Enum):
    BOOKED = "booked"
    PENDING = "pending"

@dataclass
class Transaction:
    """Represents a bank transaction."""
    transaction_id: str
    booking_date: str
    value_date: str
    amount: float
    currency: str
    creditor_name: str = ""
    debtor_name: str = ""
    remittance_info: str = ""
    bank_transaction_code: str = ""
    status: TransactionStatus = TransactionStatus.BOOKED
    
    def to_api_response(self) -> Dict:
        """Convert to Open Banking API response format."""
        response = {
            "transactionId": self.transaction_id,
            "bookingDate": self.booking_date,
            "valueDate": self.value_date,
            "transactionAmount": {
                "amount": str(self.amount),
                "currency": self.currency
            },
            "remittanceInformationUnstructured": self.remittance_info,
            "bankTransactionCode": self.bank_transaction_code
        }
        
        if self.amount > 0:
            response["debtorName"] = self.debtor_name
        else:
            response["creditorName"] = self.creditor_name
            
        return response

# Generate sample transactions
def generate_transactions(account_id: str, num_transactions: int = 50) -> List[Transaction]:
    """
    Generate realistic mock transactions for an account.
    """
    transactions = []
    
    # Common transaction patterns
    income_sources = [
        ("Employer GmbH", "Salary", 2500, 4500),
        ("Tax Office", "Tax Refund", 200, 1500),
        ("Parent", "Transfer", 100, 500),
    ]
    
    expense_types = [
        ("REWE", "Groceries", 20, 150),
        ("Amazon EU", "Online Shopping", 15, 300),
        ("Spotify", "Subscription", 9.99, 9.99),
        ("Netflix", "Subscription", 12.99, 12.99),
        ("Deutsche Bahn", "Travel", 30, 200),
        ("Shell", "Fuel", 40, 100),
        ("Restaurant", "Dining", 15, 80),
        ("Landlord", "Rent", 800, 1500),
        ("Insurance Co", "Insurance", 50, 200),
        ("Utility Provider", "Utilities", 80, 200),
    ]
    
    base_date = datetime.now()
    
    for i in range(num_transactions):
        # Random date in the past 90 days
        days_ago = random.randint(0, 90)
        tx_date = base_date - timedelta(days=days_ago)
        date_str = tx_date.strftime('%Y-%m-%d')
        
        # 20% chance of income, 80% expense
        if random.random() < 0.2:
            source = random.choice(income_sources)
            name, desc, min_amt, max_amt = source
            amount = round(random.uniform(min_amt, max_amt), 2)
            debtor = name
            creditor = ""
        else:
            expense = random.choice(expense_types)
            name, desc, min_amt, max_amt = expense
            amount = -round(random.uniform(min_amt, max_amt), 2)
            creditor = name
            debtor = ""
        
        # Determine status (recent transactions may be pending)
        status = TransactionStatus.PENDING if days_ago < 2 and random.random() < 0.3 else TransactionStatus.BOOKED
        
        tx = Transaction(
            transaction_id=f"TX{account_id[-4:]}{i:05d}",
            booking_date=date_str,
            value_date=date_str,
            amount=amount,
            currency="EUR",
            creditor_name=creditor,
            debtor_name=debtor,
            remittance_info=desc,
            bank_transaction_code="PMNT" if amount < 0 else "RCDT",
            status=status
        )
        transactions.append(tx)
    
    # Sort by date (newest first)
    transactions.sort(key=lambda x: x.booking_date, reverse=True)
    
    return transactions

# Add transactions to all accounts
for acc_id in demo_bank.accounts.keys():
    demo_bank.transactions[acc_id] = generate_transactions(acc_id, num_transactions=50)

print(f"Generated transactions for {len(demo_bank.accounts)} accounts")
print(f"Total transactions: {sum(len(txs) for txs in demo_bank.transactions.values())}")

In [ ]:
# Add transactions API to MockBank

def get_transactions(self, account_id: str, consent_id: str, 
                     date_from: str = None, date_to: str = None,
                     booking_status: str = "both") -> Dict:
    """
    API: Get account transactions.
    
    Args:
        account_id: The account identifier
        consent_id: Valid consent ID
        date_from: Start date (YYYY-MM-DD)
        date_to: End date (YYYY-MM-DD)
        booking_status: 'booked', 'pending', or 'both'
    """
    # Consent check
    if consent_id not in self.consents or self.consents[consent_id]["status"] != "valid":
        return {"error": "CONSENT_INVALID"}
    
    if account_id not in self.accounts:
        return {"error": "ACCOUNT_NOT_FOUND"}
    
    transactions = self.transactions.get(account_id, [])
    
    # Filter by date
    if date_from:
        transactions = [t for t in transactions if t.booking_date >= date_from]
    if date_to:
        transactions = [t for t in transactions if t.booking_date <= date_to]
    
    # Filter by status
    booked = [t for t in transactions if t.status == TransactionStatus.BOOKED]
    pending = [t for t in transactions if t.status == TransactionStatus.PENDING]
    
    response = {
        "account": {"iban": self.accounts[account_id].iban},
        "transactions": {}
    }
    
    if booking_status in ["booked", "both"]:
        response["transactions"]["booked"] = [t.to_api_response() for t in booked]
    if booking_status in ["pending", "both"]:
        response["transactions"]["pending"] = [t.to_api_response() for t in pending]
    
    response["_links"] = {
        "account": f"/accounts/{account_id}"
    }
    
    return response

# Add method to MockBank
MockBank.get_transactions = get_transactions

print("Transactions API added to MockBank")

In [ ]:
# Demonstrate the Transactions API

def demo_transactions_api(bank: MockBank, account_id: str, consent_id: str):
    """
    Demonstrate the transactions API endpoint.
    """
    print("\n" + "="*80)
    print(f"OPEN BANKING API: GET /accounts/{account_id}/transactions")
    print("="*80)
    
    # Calculate date range for last 30 days
    date_to = datetime.now().strftime('%Y-%m-%d')
    date_from = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    print(f"\nRequest:")
    print(f"  GET /accounts/{account_id}/transactions?dateFrom={date_from}&dateTo={date_to}&bookingStatus=both")
    print(f"  Headers:")
    print(f"    Authorization: Bearer <access_token>")
    print(f"    Consent-ID: {consent_id}")
    
    response = bank.get_transactions(account_id, consent_id, date_from, date_to, "both")
    
    # Show summary instead of full response (for readability)
    print(f"\nResponse Summary:")
    print(f"  Account IBAN: {response['account']['iban']}")
    
    booked = response['transactions'].get('booked', [])
    pending = response['transactions'].get('pending', [])
    
    print(f"  Booked transactions: {len(booked)}")
    print(f"  Pending transactions: {len(pending)}")
    
    # Show first 5 booked transactions
    print(f"\n  Recent Booked Transactions (first 5):")
    print(f"  {'-'*60}")
    for tx in booked[:5]:
        amount = float(tx['transactionAmount']['amount'])
        name = tx.get('creditorName', tx.get('debtorName', 'Unknown'))
        desc = tx.get('remittanceInformationUnstructured', '')
        sign = '+' if amount > 0 else ''
        print(f"  {tx['bookingDate']}  {sign}{amount:>10.2f} EUR  {name[:20]:<20} {desc}")
    
    # Calculate totals
    total_credits = sum(float(t['transactionAmount']['amount']) for t in booked if float(t['transactionAmount']['amount']) > 0)
    total_debits = sum(float(t['transactionAmount']['amount']) for t in booked if float(t['transactionAmount']['amount']) < 0)
    
    print(f"\n  Period Summary:")
    print(f"    Total Credits: +{total_credits:,.2f} EUR")
    print(f"    Total Debits:  {total_debits:,.2f} EUR")
    print(f"    Net Flow:      {total_credits + total_debits:,.2f} EUR")

# Demo transactions for first account
demo_transactions_api(demo_bank, "ACC000001", consent_id)

## Section 5: Simulate Payment Initiation API

The **Payment Initiation Service** (PIS) allows third parties to initiate payments on behalf of users. This is powerful but requires careful security controls.

### Payment Types in Open Banking

| Payment Type | Description | Settlement |
|-------------|-------------|------------|
| **SEPA Credit Transfer** | Standard EUR transfer | 1 business day |
| **SEPA Instant** | Real-time EUR transfer | < 10 seconds |
| **Domestic Transfer** | Same country, any currency | 1-2 days |
| **Cross-border** | International transfer | 2-5 days |

In [ ]:
# Payment data structures

class PaymentStatus(Enum):
    RCVD = "Received"  # Payment received by bank
    ACTC = "AcceptedTechnicalValidation"  # Technical validation OK
    ACCP = "AcceptedCustomerProfile"  # Customer profile validation OK
    ACWC = "AcceptedWithChange"  # Accepted with modifications
    ACSC = "AcceptedSettlementCompleted"  # Settlement complete
    RJCT = "Rejected"  # Payment rejected
    PDNG = "Pending"  # Awaiting further action
    CANC = "Cancelled"  # Cancelled

@dataclass
class PaymentRequest:
    """Represents a payment initiation request."""
    payment_id: str
    debtor_account: str  # IBAN of payer
    creditor_account: str  # IBAN of payee
    creditor_name: str
    amount: float
    currency: str = "EUR"
    remittance_info: str = ""
    requested_execution_date: str = None
    status: PaymentStatus = PaymentStatus.RCVD
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_api_response(self) -> Dict:
        return {
            "paymentId": self.payment_id,
            "transactionStatus": self.status.name,
            "debtorAccount": {"iban": self.debtor_account},
            "creditorAccount": {"iban": self.creditor_account},
            "creditorName": self.creditor_name,
            "instructedAmount": {
                "amount": str(self.amount),
                "currency": self.currency
            },
            "remittanceInformationUnstructured": self.remittance_info,
            "_links": {
                "self": f"/payments/sepa-credit-transfers/{self.payment_id}",
                "status": f"/payments/sepa-credit-transfers/{self.payment_id}/status"
            }
        }

# Add payments storage to MockBank
demo_bank.payments = {}

print("Payment structures defined.")
print("\nPayment Status Flow:")
print("  RCVD -> ACTC -> ACCP -> ACSC (success)")
print("  RCVD -> RJCT (rejection)")
print("  Any stage -> CANC (cancellation)")

In [ ]:
# Add Payment Initiation API to MockBank

def initiate_payment(self, consent_id: str, debtor_iban: str, 
                     creditor_iban: str, creditor_name: str,
                     amount: float, currency: str = "EUR",
                     remittance_info: str = "") -> Dict:
    """
    API: Initiate a SEPA credit transfer payment.
    
    This simulates the payment initiation process including:
    - Consent validation
    - Account validation
    - Balance check
    - Payment creation
    """
    # Step 1: Validate consent
    if consent_id not in self.consents:
        return {"error": "CONSENT_INVALID", "message": "Consent not found"}
    
    consent = self.consents[consent_id]
    if consent["status"] != "valid":
        return {"error": "CONSENT_EXPIRED"}
    
    # Step 2: Find debtor account
    debtor_account = None
    for acc in self.accounts.values():
        if acc.iban == debtor_iban:
            debtor_account = acc
            break
    
    if not debtor_account:
        return {
            "error": "RESOURCE_UNKNOWN",
            "message": "Debtor account not found"
        }
    
    # Step 3: Check account status
    if debtor_account.status != AccountStatus.ENABLED:
        return {
            "error": "ACCOUNT_BLOCKED",
            "message": "Account is not enabled for payments"
        }
    
    # Step 4: Check balance (simplified)
    available_balance = debtor_account.balances[0].amount if debtor_account.balances else 0
    if available_balance < amount:
        return {
            "error": "INSUFFICIENT_FUNDS",
            "message": f"Available: {available_balance}, Required: {amount}"
        }
    
    # Step 5: Create payment
    payment_id = f"PAY-{uuid.uuid4().hex[:12].upper()}"
    
    payment = PaymentRequest(
        payment_id=payment_id,
        debtor_account=debtor_iban,
        creditor_account=creditor_iban,
        creditor_name=creditor_name,
        amount=amount,
        currency=currency,
        remittance_info=remittance_info,
        status=PaymentStatus.RCVD
    )
    
    self.payments[payment_id] = payment
    
    # Step 6: Simulate processing (move to ACTC)
    payment.status = PaymentStatus.ACTC
    
    return {
        "transactionStatus": payment.status.name,
        "paymentId": payment_id,
        "_links": {
            "self": f"/payments/sepa-credit-transfers/{payment_id}",
            "status": f"/payments/sepa-credit-transfers/{payment_id}/status",
            "scaRedirect": f"/authorize/{payment_id}"  # Would redirect to bank's SCA
        }
    }

def get_payment_status(self, payment_id: str, consent_id: str) -> Dict:
    """
    API: Get payment status.
    """
    if payment_id not in self.payments:
        return {"error": "PAYMENT_NOT_FOUND"}
    
    payment = self.payments[payment_id]
    return {
        "transactionStatus": payment.status.name,
        "paymentId": payment_id
    }

def authorize_payment(self, payment_id: str, sca_method: str = "SMS_OTP") -> Dict:
    """
    Simulate SCA (Strong Customer Authentication) for payment.
    In reality, this would involve redirect to bank and 2FA.
    """
    if payment_id not in self.payments:
        return {"error": "PAYMENT_NOT_FOUND"}
    
    payment = self.payments[payment_id]
    
    # Simulate successful SCA
    payment.status = PaymentStatus.ACCP
    
    # After a short delay, complete the payment
    payment.status = PaymentStatus.ACSC
    
    return {
        "transactionStatus": payment.status.name,
        "paymentId": payment_id,
        "scaStatus": "finalised",
        "message": "Payment authorized and executed successfully"
    }

# Add methods to MockBank
MockBank.initiate_payment = initiate_payment
MockBank.get_payment_status = get_payment_status
MockBank.authorize_payment = authorize_payment

print("Payment Initiation API added to MockBank")

In [ ]:
# Demonstrate the Payment Initiation API

def demo_payment_initiation(bank: MockBank, consent_id: str):
    """
    Demonstrate the complete payment initiation flow.
    """
    print("\n" + "="*80)
    print("PAYMENT INITIATION SERVICE (PIS) DEMONSTRATION")
    print("="*80)
    
    # Get debtor's IBAN
    debtor_account = bank.accounts["ACC000001"]
    debtor_iban = debtor_account.iban
    
    # Create a payment request
    creditor_iban = "DE89370400440532013000"  # Sample IBAN
    
    print("\n[STEP 1] POST /payments/sepa-credit-transfers")
    print("-" * 50)
    
    request_body = {
        "debtorAccount": {"iban": debtor_iban},
        "creditorAccount": {"iban": creditor_iban},
        "creditorName": "Max Mustermann",
        "instructedAmount": {
            "amount": "100.00",
            "currency": "EUR"
        },
        "remittanceInformationUnstructured": "Invoice #12345"
    }
    
    print("Request Body:")
    print(json.dumps(request_body, indent=2))
    
    # Initiate the payment
    response = bank.initiate_payment(
        consent_id=consent_id,
        debtor_iban=debtor_iban,
        creditor_iban=creditor_iban,
        creditor_name="Max Mustermann",
        amount=100.00,
        currency="EUR",
        remittance_info="Invoice #12345"
    )
    
    print("\nResponse:")
    print(json.dumps(response, indent=2))
    
    if "error" in response:
        print(f"\nPayment initiation failed: {response['error']}")
        return
    
    payment_id = response["paymentId"]
    
    # Step 2: Check status
    print(f"\n[STEP 2] GET /payments/sepa-credit-transfers/{payment_id}/status")
    print("-" * 50)
    
    status_response = bank.get_payment_status(payment_id, consent_id)
    print(f"Status: {status_response['transactionStatus']}")
    print("(Payment awaiting Strong Customer Authentication)")
    
    # Step 3: Simulate SCA
    print(f"\n[STEP 3] Strong Customer Authentication (SCA)")
    print("-" * 50)
    print("In a real scenario, the user would be redirected to the bank's website")
    print("to authenticate using 2FA (e.g., mobile app confirmation, SMS OTP).")
    print("")
    print("SCA Methods typically include:")
    print("  - Mobile banking app push notification")
    print("  - SMS One-Time Password")
    print("  - Hardware token")
    print("  - Biometric authentication")
    print("")
    print("Simulating successful SCA...")
    
    auth_response = bank.authorize_payment(payment_id)
    print(f"\nAuthorization Response:")
    print(json.dumps(auth_response, indent=2))
    
    # Step 4: Final status
    print(f"\n[STEP 4] Payment Complete")
    print("-" * 50)
    final_status = bank.get_payment_status(payment_id, consent_id)
    print(f"Final Status: {final_status['transactionStatus']}")
    
    print("\n" + "="*80)
    print("PAYMENT FLOW SUMMARY")
    print("="*80)
    print(f"""
    1. TPP (Third Party Provider) initiates payment request
    2. Bank validates request (RCVD -> ACTC)
    3. User authenticates via SCA (ACTC -> ACCP)
    4. Bank executes payment (ACCP -> ACSC)
    5. Funds transferred to creditor
    
    Payment ID: {payment_id}
    Amount: 100.00 EUR
    From: {debtor_iban}
    To: {creditor_iban}
    Status: COMPLETED
    """)

# Run the demo
demo_payment_initiation(demo_bank, consent_id)

## Section 6: OAuth 2.0 Flow Simulation

Open Banking uses **OAuth 2.0** for authorization. This is the standard protocol that allows users to grant limited access to their data without sharing passwords.

### OAuth 2.0 in Open Banking

```
+--------+                               +---------------+
|        |---(1) Authorization Request-->|   Resource    |
|        |                               |     Owner     |
|        |<--(2) Authorization Grant-----|   (User)      |
|        |                               +---------------+
|        |
| Client |                               +---------------+
| (TPP)  |---(3) Authorization Grant---->| Authorization |
|        |                               |     Server    |
|        |<--(4) Access Token------------| (Bank)        |
|        |                               +---------------+
|        |
|        |                               +---------------+
|        |---(5) Access Token----------->|    Resource   |
|        |                               |     Server    |
|        |<--(6) Protected Resource------|  (Bank API)   |
+--------+                               +---------------+
```

In [ ]:
# OAuth 2.0 simulation classes

@dataclass
class OAuthClient:
    """Represents a registered TPP (Third Party Provider)."""
    client_id: str
    client_secret: str
    client_name: str
    redirect_uris: List[str]
    scope: List[str]  # e.g., ['accounts', 'payments']
    tpp_id: str  # TPP registration number

@dataclass
class AuthorizationCode:
    """Represents an authorization code."""
    code: str
    client_id: str
    user_id: str
    scope: List[str]
    redirect_uri: str
    expires_at: datetime
    used: bool = False

@dataclass  
class AccessToken:
    """Represents an access token."""
    token: str
    token_type: str = "Bearer"
    client_id: str = ""
    user_id: str = ""
    scope: List[str] = field(default_factory=list)
    expires_at: datetime = None
    refresh_token: str = ""

class OAuthServer:
    """
    Simulates an OAuth 2.0 Authorization Server for Open Banking.
    """
    
    def __init__(self, bank: MockBank):
        self.bank = bank
        self.clients: Dict[str, OAuthClient] = {}
        self.authorization_codes: Dict[str, AuthorizationCode] = {}
        self.access_tokens: Dict[str, AccessToken] = {}
        self.refresh_tokens: Dict[str, str] = {}  # Maps refresh_token to access_token
    
    def register_client(self, client_name: str, redirect_uris: List[str], 
                        scope: List[str], tpp_id: str) -> OAuthClient:
        """
        Register a new TPP client. In reality, this involves regulatory approval.
        """
        client_id = f"client_{secrets.token_hex(8)}"
        client_secret = secrets.token_hex(32)
        
        client = OAuthClient(
            client_id=client_id,
            client_secret=client_secret,
            client_name=client_name,
            redirect_uris=redirect_uris,
            scope=scope,
            tpp_id=tpp_id
        )
        
        self.clients[client_id] = client
        return client
    
    def authorize(self, client_id: str, redirect_uri: str, scope: List[str],
                  state: str, user_id: str) -> Tuple[str, str]:
        """
        Step 1-2: Authorization endpoint.
        User grants permission, returns authorization code.
        """
        # Validate client
        if client_id not in self.clients:
            return None, "invalid_client"
        
        client = self.clients[client_id]
        
        # Validate redirect URI
        if redirect_uri not in client.redirect_uris:
            return None, "invalid_redirect_uri"
        
        # Validate scope
        if not all(s in client.scope for s in scope):
            return None, "invalid_scope"
        
        # Generate authorization code
        code = secrets.token_urlsafe(32)
        
        auth_code = AuthorizationCode(
            code=code,
            client_id=client_id,
            user_id=user_id,
            scope=scope,
            redirect_uri=redirect_uri,
            expires_at=datetime.now() + timedelta(minutes=5)  # Short-lived
        )
        
        self.authorization_codes[code] = auth_code
        
        return code, state
    
    def token(self, grant_type: str, code: str = None, client_id: str = None,
              client_secret: str = None, redirect_uri: str = None,
              refresh_token: str = None) -> Dict:
        """
        Step 3-4: Token endpoint.
        Exchange authorization code for access token.
        """
        if grant_type == "authorization_code":
            # Validate client credentials
            if client_id not in self.clients:
                return {"error": "invalid_client"}
            
            client = self.clients[client_id]
            if client.client_secret != client_secret:
                return {"error": "invalid_client"}
            
            # Validate authorization code
            if code not in self.authorization_codes:
                return {"error": "invalid_grant"}
            
            auth_code = self.authorization_codes[code]
            
            if auth_code.used:
                return {"error": "invalid_grant", "error_description": "Code already used"}
            
            if auth_code.expires_at < datetime.now():
                return {"error": "invalid_grant", "error_description": "Code expired"}
            
            if auth_code.client_id != client_id:
                return {"error": "invalid_grant"}
            
            if auth_code.redirect_uri != redirect_uri:
                return {"error": "invalid_grant"}
            
            # Mark code as used
            auth_code.used = True
            
            # Generate tokens
            access_token = secrets.token_urlsafe(32)
            refresh_tok = secrets.token_urlsafe(32)
            
            token_obj = AccessToken(
                token=access_token,
                client_id=client_id,
                user_id=auth_code.user_id,
                scope=auth_code.scope,
                expires_at=datetime.now() + timedelta(hours=1),
                refresh_token=refresh_tok
            )
            
            self.access_tokens[access_token] = token_obj
            self.refresh_tokens[refresh_tok] = access_token
            
            # Create consent in bank
            consent_id = f"consent-{uuid.uuid4().hex[:8]}"
            self.bank.consents[consent_id] = {
                "status": "valid",
                "customer_id": auth_code.user_id,
                "access_token": access_token,
                "scope": auth_code.scope,
                "created": datetime.now().isoformat(),
                "expires": (datetime.now() + timedelta(days=90)).isoformat()
            }
            
            return {
                "access_token": access_token,
                "token_type": "Bearer",
                "expires_in": 3600,
                "refresh_token": refresh_tok,
                "scope": " ".join(auth_code.scope),
                "consent_id": consent_id
            }
        
        elif grant_type == "refresh_token":
            if refresh_token not in self.refresh_tokens:
                return {"error": "invalid_grant"}
            
            old_access_token = self.refresh_tokens[refresh_token]
            old_token_obj = self.access_tokens.get(old_access_token)
            
            if not old_token_obj:
                return {"error": "invalid_grant"}
            
            # Generate new tokens
            new_access_token = secrets.token_urlsafe(32)
            new_refresh_token = secrets.token_urlsafe(32)
            
            new_token_obj = AccessToken(
                token=new_access_token,
                client_id=old_token_obj.client_id,
                user_id=old_token_obj.user_id,
                scope=old_token_obj.scope,
                expires_at=datetime.now() + timedelta(hours=1),
                refresh_token=new_refresh_token
            )
            
            # Invalidate old tokens
            del self.access_tokens[old_access_token]
            del self.refresh_tokens[refresh_token]
            
            # Store new tokens
            self.access_tokens[new_access_token] = new_token_obj
            self.refresh_tokens[new_refresh_token] = new_access_token
            
            return {
                "access_token": new_access_token,
                "token_type": "Bearer",
                "expires_in": 3600,
                "refresh_token": new_refresh_token,
                "scope": " ".join(old_token_obj.scope)
            }
        
        return {"error": "unsupported_grant_type"}

# Create OAuth server
oauth_server = OAuthServer(demo_bank)

print("OAuth 2.0 Server initialized")

In [ ]:
# Demonstrate the complete OAuth 2.0 flow

def demo_oauth_flow(oauth: OAuthServer, user_id: str):
    """
    Demonstrate the complete OAuth 2.0 Authorization Code flow.
    """
    print("\n" + "="*80)
    print("OAuth 2.0 AUTHORIZATION CODE FLOW")
    print("="*80)
    
    # Step 0: Register TPP
    print("\n[STEP 0] TPP Registration")
    print("-" * 50)
    print("In reality, TPPs must be registered with financial regulators.")
    print("They receive a TPP ID and are added to a public register.")
    
    client = oauth.register_client(
        client_name="FinanceApp Pro",
        redirect_uris=["https://financeapp.example/callback"],
        scope=["accounts", "payments"],
        tpp_id="TPP-2024-001234"
    )
    
    print(f"\nRegistered TPP: {client.client_name}")
    print(f"  Client ID: {client.client_id}")
    print(f"  TPP ID: {client.tpp_id}")
    print(f"  Allowed Scope: {', '.join(client.scope)}")
    
    # Step 1: Authorization Request
    print("\n[STEP 1] Authorization Request")
    print("-" * 50)
    
    state = secrets.token_urlsafe(16)  # CSRF protection
    auth_url = f"""
    GET /authorize?
        response_type=code&
        client_id={client.client_id}&
        redirect_uri=https://financeapp.example/callback&
        scope=accounts payments&
        state={state}
    """
    print(f"Authorization URL:{auth_url}")
    
    # Step 2: User authenticates and grants consent
    print("\n[STEP 2] User Authentication & Consent")
    print("-" * 50)
    print("""The user is shown a consent screen:
    
    +------------------------------------------+
    |     Demo Digital Bank                    |
    +------------------------------------------+
    |                                          |
    |  FinanceApp Pro is requesting access to: |
    |                                          |
    |  [x] View your account balances          |
    |  [x] View your transaction history       |
    |  [x] Initiate payments on your behalf    |
    |                                          |
    |  This access will expire in 90 days.     |
    |                                          |
    |  [ Deny ]              [ Allow ]         |
    +------------------------------------------+
    """)
    
    print("User clicks 'Allow'...")
    
    code, returned_state = oauth.authorize(
        client_id=client.client_id,
        redirect_uri="https://financeapp.example/callback",
        scope=["accounts", "payments"],
        state=state,
        user_id=user_id
    )
    
    print(f"\nUser redirected to:")
    print(f"  https://financeapp.example/callback?code={code[:20]}...&state={state}")
    
    # Verify state matches (CSRF protection)
    assert state == returned_state, "State mismatch - possible CSRF attack!"
    print("\n  State verified (CSRF protection)")
    
    # Step 3: Exchange code for tokens
    print("\n[STEP 3] Token Exchange")
    print("-" * 50)
    print("TPP exchanges authorization code for access token:")
    print(f"""
    POST /token
    Content-Type: application/x-www-form-urlencoded
    
    grant_type=authorization_code&
    code={code[:20]}...&
    client_id={client.client_id}&
    client_secret=***REDACTED***&
    redirect_uri=https://financeapp.example/callback
    """)
    
    token_response = oauth.token(
        grant_type="authorization_code",
        code=code,
        client_id=client.client_id,
        client_secret=client.client_secret,
        redirect_uri="https://financeapp.example/callback"
    )
    
    print("Token Response:")
    # Redact sensitive values for display
    display_response = token_response.copy()
    if "access_token" in display_response:
        display_response["access_token"] = display_response["access_token"][:20] + "..."
    if "refresh_token" in display_response:
        display_response["refresh_token"] = display_response["refresh_token"][:20] + "..."
    print(json.dumps(display_response, indent=2))
    
    # Step 4: Use access token to call API
    print("\n[STEP 4] Access Protected Resource")
    print("-" * 50)
    print("TPP can now access the user's accounts:")
    print(f"""
    GET /accounts
    Authorization: Bearer {token_response['access_token'][:20]}...
    Consent-ID: {token_response['consent_id']}
    """)
    
    # Make API call
    accounts = oauth.bank.get_accounts(user_id, token_response['consent_id'])
    
    print(f"Response: {len(accounts.get('accounts', []))} accounts retrieved")
    for acc in accounts.get('accounts', [])[:2]:
        print(f"  - {acc['name']}: {acc['iban']}")
    
    # Step 5: Token refresh
    print("\n[STEP 5] Token Refresh")
    print("-" * 50)
    print("When the access token expires, use refresh token to get a new one:")
    
    refresh_response = oauth.token(
        grant_type="refresh_token",
        refresh_token=token_response['refresh_token']
    )
    
    print("New access token obtained (old token invalidated)")
    
    print("\n" + "="*80)
    print("OAuth 2.0 FLOW COMPLETE")
    print("="*80)
    print("""
    Key Security Features:
    
    1. Authorization Code: Short-lived, single-use, exchanged server-to-server
    2. Access Token: Bearer token with limited lifetime (typically 1 hour)
    3. Refresh Token: Longer-lived, used to obtain new access tokens
    4. State Parameter: CSRF protection
    5. PKCE: Additional security for public clients (not shown here)
    6. Consent: User explicitly grants permissions
    7. Scope: Limits what the TPP can access
    """)
    
    return token_response

# Run the OAuth demo
token_response = demo_oauth_flow(oauth_server, "CUST0001")

## Section 7: Third-Party Provider Simulation

Now let's build an **Account Aggregator** - a Third Party Provider (TPP) that connects to multiple banks and provides a unified view of all accounts.

This is one of the most valuable Open Banking use cases:
- Personal Finance Management (PFM) apps
- Lending decisions based on complete financial picture
- Business accounting software integration

In [ ]:
# Create additional mock banks

# Bank 2: Traditional Bank
traditional_bank = MockBank("Traditional Savings Bank", "TRADBANKXXX")
traditional_bank = create_sample_accounts(traditional_bank, num_customers=3)

# Bank 3: Digital-only Bank
digital_bank = MockBank("Neon Digital Bank", "NEONBANKXXX")
digital_bank = create_sample_accounts(digital_bank, num_customers=3)

# Add transactions to new banks
for bank in [traditional_bank, digital_bank]:
    for acc_id in bank.accounts.keys():
        bank.transactions[acc_id] = generate_transactions(acc_id, num_transactions=30)

print("Created additional banks:")
print(f"  - {traditional_bank.bank_name} ({len(traditional_bank.accounts)} accounts)")
print(f"  - {digital_bank.bank_name} ({len(digital_bank.accounts)} accounts)")

In [ ]:
# Build the Account Aggregator TPP

class AccountAggregator:
    """
    A Third Party Provider that aggregates accounts from multiple banks.
    This simulates apps like Mint, Plaid, or TrueLayer.
    """
    
    def __init__(self, name: str, tpp_id: str):
        self.name = name
        self.tpp_id = tpp_id
        self.connected_banks: Dict[str, Dict] = {}  # bank_name -> {bank, consent_id, user_id}
        self.users: Dict[str, Dict] = {}  # user_id -> {bank_connections: [...]}
    
    def connect_bank(self, user_id: str, bank: MockBank, customer_id: str) -> str:
        """
        Connect a user's bank account (simulates OAuth flow completion).
        """
        # In reality, this would involve the full OAuth flow
        # For simulation, we create a consent directly
        consent_id = f"agg-consent-{uuid.uuid4().hex[:8]}"
        
        bank.consents[consent_id] = {
            "status": "valid",
            "customer_id": customer_id,
            "tpp_id": self.tpp_id,
            "created": datetime.now().isoformat(),
            "expires": (datetime.now() + timedelta(days=90)).isoformat()
        }
        
        # Store connection
        connection = {
            "bank": bank,
            "bank_name": bank.bank_name,
            "consent_id": consent_id,
            "customer_id": customer_id,
            "connected_at": datetime.now().isoformat()
        }
        
        if user_id not in self.users:
            self.users[user_id] = {"bank_connections": []}
        
        self.users[user_id]["bank_connections"].append(connection)
        
        return consent_id
    
    def get_all_accounts(self, user_id: str) -> Dict:
        """
        Get aggregated view of all accounts across connected banks.
        """
        if user_id not in self.users:
            return {"error": "USER_NOT_FOUND"}
        
        all_accounts = []
        
        for connection in self.users[user_id]["bank_connections"]:
            bank = connection["bank"]
            consent_id = connection["consent_id"]
            customer_id = connection["customer_id"]
            
            # Fetch accounts from this bank
            response = bank.get_accounts(customer_id, consent_id)
            
            if "accounts" in response:
                for account in response["accounts"]:
                    account["_bank"] = connection["bank_name"]
                    account["_bic"] = bank.bic
                    all_accounts.append(account)
        
        return {
            "accounts": all_accounts,
            "totalBanksConnected": len(self.users[user_id]["bank_connections"]),
            "totalAccounts": len(all_accounts),
            "_aggregator": self.name
        }
    
    def get_total_balance(self, user_id: str) -> Dict:
        """
        Calculate total balance across all connected accounts.
        """
        if user_id not in self.users:
            return {"error": "USER_NOT_FOUND"}
        
        total_by_currency = {}
        account_details = []
        
        for connection in self.users[user_id]["bank_connections"]:
            bank = connection["bank"]
            consent_id = connection["consent_id"]
            customer_id = connection["customer_id"]
            
            # Get accounts
            accounts_response = bank.get_accounts(customer_id, consent_id)
            
            if "accounts" in accounts_response:
                for account in accounts_response["accounts"]:
                    # Get balance for each account
                    balance_response = bank.get_balances(account["accountId"], consent_id)
                    
                    if "balances" in balance_response:
                        for bal in balance_response["balances"]:
                            if bal["balanceType"] == "ClosingAvailable":
                                currency = bal["balanceAmount"]["currency"]
                                amount = float(bal["balanceAmount"]["amount"])
                                
                                if currency not in total_by_currency:
                                    total_by_currency[currency] = 0
                                total_by_currency[currency] += amount
                                
                                account_details.append({
                                    "bank": connection["bank_name"],
                                    "account": account["name"],
                                    "balance": amount,
                                    "currency": currency
                                })
        
        return {
            "totalBalances": [
                {"currency": curr, "amount": amt} 
                for curr, amt in total_by_currency.items()
            ],
            "accountBreakdown": account_details,
            "calculatedAt": datetime.now().isoformat()
        }
    
    def get_all_transactions(self, user_id: str, days: int = 30) -> Dict:
        """
        Get combined transaction history across all banks.
        """
        if user_id not in self.users:
            return {"error": "USER_NOT_FOUND"}
        
        all_transactions = []
        date_from = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
        
        for connection in self.users[user_id]["bank_connections"]:
            bank = connection["bank"]
            consent_id = connection["consent_id"]
            customer_id = connection["customer_id"]
            
            accounts_response = bank.get_accounts(customer_id, consent_id)
            
            if "accounts" in accounts_response:
                for account in accounts_response["accounts"]:
                    tx_response = bank.get_transactions(
                        account["accountId"], consent_id, date_from=date_from
                    )
                    
                    if "transactions" in tx_response:
                        for tx in tx_response["transactions"].get("booked", []):
                            tx["_bank"] = connection["bank_name"]
                            tx["_accountName"] = account["name"]
                            all_transactions.append(tx)
        
        # Sort all transactions by date
        all_transactions.sort(key=lambda x: x.get("bookingDate", ""), reverse=True)
        
        return {
            "transactions": all_transactions,
            "totalTransactions": len(all_transactions),
            "dateRange": {
                "from": date_from,
                "to": datetime.now().strftime('%Y-%m-%d')
            }
        }

# Create aggregator
aggregator = AccountAggregator("FinanceHub Aggregator", "TPP-AGG-001234")

print(f"Created Account Aggregator: {aggregator.name}")
print(f"TPP ID: {aggregator.tpp_id}")

In [ ]:
# Demonstrate the Account Aggregator

def demo_account_aggregator(aggregator: AccountAggregator, banks: List[MockBank]):
    """
    Demonstrate the account aggregation functionality.
    """
    print("\n" + "="*80)
    print("ACCOUNT AGGREGATOR DEMONSTRATION")
    print("="*80)
    
    user_id = "aggregator-user-001"
    
    # Connect multiple banks
    print("\n[STEP 1] Connecting Banks")
    print("-" * 50)
    
    for i, bank in enumerate(banks):
        # Use first customer from each bank
        customer_id = list(bank.customers.keys())[0]
        consent = aggregator.connect_bank(user_id, bank, customer_id)
        print(f"  Connected: {bank.bank_name}")
        print(f"    Customer: {bank.customers[customer_id]['name']}")
        print(f"    Consent: {consent}")
    
    # Get aggregated accounts
    print("\n[STEP 2] Aggregated Account View")
    print("-" * 50)
    
    accounts = aggregator.get_all_accounts(user_id)
    print(f"\nTotal banks connected: {accounts['totalBanksConnected']}")
    print(f"Total accounts: {accounts['totalAccounts']}")
    print("\nAccounts by Bank:")
    
    current_bank = None
    for acc in accounts['accounts']:
        if acc['_bank'] != current_bank:
            current_bank = acc['_bank']
            print(f"\n  {current_bank}:")
        print(f"    - {acc['name']} ({acc['accountType']})")
        print(f"      IBAN: {acc['iban']}")
    
    # Get total balance
    print("\n[STEP 3] Total Balance Summary")
    print("-" * 50)
    
    balances = aggregator.get_total_balance(user_id)
    
    print("\nNet Worth Summary:")
    for total in balances['totalBalances']:
        print(f"  {total['currency']}: {total['amount']:,.2f}")
    
    print("\nBreakdown by Account:")
    for detail in balances['accountBreakdown']:
        print(f"  {detail['bank'][:20]:<20} {detail['account'][:25]:<25} {detail['balance']:>12,.2f} {detail['currency']}")
    
    # Get combined transactions
    print("\n[STEP 4] Combined Transaction History (Last 30 days)")
    print("-" * 50)
    
    tx_data = aggregator.get_all_transactions(user_id, days=30)
    
    print(f"\nTotal transactions: {tx_data['totalTransactions']}")
    print(f"Date range: {tx_data['dateRange']['from']} to {tx_data['dateRange']['to']}")
    
    # Show recent transactions
    print("\nMost Recent Transactions:")
    print(f"{'Date':<12} {'Bank':<20} {'Account':<20} {'Amount':>12} {'Description'}")
    print("-" * 80)
    
    for tx in tx_data['transactions'][:10]:
        amount = float(tx['transactionAmount']['amount'])
        desc = tx.get('remittanceInformationUnstructured', '')[:15]
        sign = '+' if amount > 0 else ''
        print(f"{tx['bookingDate']:<12} {tx['_bank'][:18]:<20} {tx['_accountName'][:18]:<20} {sign}{amount:>11.2f} {desc}")
    
    # Calculate spending insights
    print("\n[STEP 5] Spending Insights")
    print("-" * 50)
    
    total_income = sum(
        float(tx['transactionAmount']['amount']) 
        for tx in tx_data['transactions'] 
        if float(tx['transactionAmount']['amount']) > 0
    )
    total_expenses = sum(
        float(tx['transactionAmount']['amount']) 
        for tx in tx_data['transactions'] 
        if float(tx['transactionAmount']['amount']) < 0
    )
    
    print(f"\n  Total Income:   +{total_income:,.2f} EUR")
    print(f"  Total Expenses: {total_expenses:,.2f} EUR")
    print(f"  Net Cash Flow:  {total_income + total_expenses:,.2f} EUR")
    
    print("\n" + "="*80)
    print("AGGREGATION COMPLETE")
    print("="*80)
    print("""
    Key Benefits of Account Aggregation:
    
    1. Single View: See all accounts across banks in one place
    2. Net Worth Tracking: Know your total financial position
    3. Cash Flow Analysis: Track income vs. expenses across accounts
    4. Better Lending: Lenders can make better decisions with full picture
    5. Budgeting: Categorize and track spending across all accounts
    6. Alerts: Get notified of unusual activity anywhere
    """)

# Run the demonstration
demo_account_aggregator(aggregator, [demo_bank, traditional_bank, digital_bank])

## Section 8: Challenge Exercises

Test your understanding of Open Banking concepts with these hands-on challenges!

### Challenge 1: Implement Consent Management

Open Banking requires explicit user consent that can be revoked. Implement a consent management system that:
1. Allows users to view all active consents
2. Shows what data each TPP can access
3. Allows users to revoke consent

In [ ]:
# Challenge 1: Your code here

class ConsentManager:
    """
    Manage user consents for Open Banking access.
    """
    
    def __init__(self, bank: MockBank):
        self.bank = bank
    
    def list_consents(self, customer_id: str) -> List[Dict]:
        """
        List all active consents for a customer.
        
        TODO: Implement this method
        Hints:
        - Filter self.bank.consents by customer_id
        - Return list of consents with TPP info, scope, expiry date
        """
        pass  # Replace with your implementation
    
    def revoke_consent(self, consent_id: str, customer_id: str) -> Dict:
        """
        Revoke a specific consent.
        
        TODO: Implement this method
        Hints:
        - Verify the consent belongs to the customer
        - Set status to 'revoked'
        - Return confirmation or error
        """
        pass  # Replace with your implementation
    
    def get_consent_details(self, consent_id: str) -> Dict:
        """
        Get detailed information about a consent.
        
        TODO: Implement this method
        """
        pass  # Replace with your implementation

# Test your implementation
# consent_mgr = ConsentManager(demo_bank)
# consents = consent_mgr.list_consents("CUST0001")
# print(f"Active consents: {len(consents)}")

### Challenge 2: Build a Spending Categorizer

Use transaction data to categorize spending. This is a common feature in personal finance apps.

In [ ]:
# Challenge 2: Your code here

def categorize_transactions(transactions: List[Dict]) -> Dict[str, List[Dict]]:
    """
    Categorize transactions based on merchant/description.
    
    Args:
        transactions: List of transaction dictionaries from the API
    
    Returns:
        Dictionary mapping categories to transactions
    
    TODO: Implement this function
    Hints:
    - Define category rules based on creditor name or description
    - Categories might include: Groceries, Entertainment, Transport, 
      Utilities, Shopping, Dining, Income, Transfers
    - Handle unknown/uncategorized transactions
    """
    categories = {
        "Groceries": [],
        "Entertainment": [],
        "Transport": [],
        "Utilities": [],
        "Shopping": [],
        "Dining": [],
        "Income": [],
        "Other": []
    }
    
    # TODO: Implement categorization logic
    pass  # Replace with your implementation
    
    return categories

def spending_summary(categories: Dict[str, List[Dict]]) -> None:
    """
    Print a spending summary by category.
    
    TODO: Implement this function
    """
    pass  # Replace with your implementation

# Test your implementation
# tx_data = aggregator.get_all_transactions("aggregator-user-001", days=30)
# categories = categorize_transactions(tx_data['transactions'])
# spending_summary(categories)

### Challenge 3: Implement Payment Validation

Before initiating a payment, TPPs should validate the payment details. Implement comprehensive validation.

In [ ]:
# Challenge 3: Your code here

def validate_iban(iban: str) -> Tuple[bool, str]:
    """
    Validate an IBAN (International Bank Account Number).
    
    Args:
        iban: The IBAN to validate
    
    Returns:
        Tuple of (is_valid, error_message)
    
    TODO: Implement IBAN validation
    Hints:
    - Check length (varies by country)
    - Validate country code
    - Implement mod-97 check digit validation
    """
    pass  # Replace with your implementation

def validate_payment_request(debtor_iban: str, creditor_iban: str,
                             amount: float, currency: str) -> Dict:
    """
    Validate a payment initiation request.
    
    Returns:
        Dict with 'valid' boolean and 'errors' list
    
    TODO: Implement comprehensive validation
    Check:
    - Both IBANs are valid
    - Amount is positive and within limits
    - Currency is supported
    - Debtor and creditor are different
    """
    errors = []
    
    # TODO: Implement validation logic
    pass  # Replace with your implementation
    
    return {
        "valid": len(errors) == 0,
        "errors": errors
    }

# Test your implementation
# result = validate_payment_request(
#     debtor_iban="DE89370400440532013000",
#     creditor_iban="FR7630006000011234567890189",
#     amount=100.00,
#     currency="EUR"
# )
# print(f"Valid: {result['valid']}")
# if result['errors']:
#     print(f"Errors: {result['errors']}")

### Challenge 4: Build a Simple Credit Score Calculator

Use aggregated financial data to calculate a simple credit score. This simulates how lenders use Open Banking data.

In [ ]:
# Challenge 4: Your code here

def calculate_credit_score(balances: Dict, transactions: List[Dict]) -> Dict:
    """
    Calculate a simple credit score based on financial data.
    
    Args:
        balances: Balance summary from aggregator
        transactions: Transaction history from aggregator
    
    Returns:
        Dict with score and factors
    
    TODO: Implement scoring logic
    Consider factors like:
    - Average balance (higher = better)
    - Income regularity (consistent salary = better)
    - Spending patterns (within income = better)
    - Number of accounts (diversified = better)
    - Overdraft usage (less = better)
    
    Score range: 300-850 (like FICO)
    """
    score = 550  # Base score
    factors = []
    
    # TODO: Implement scoring algorithm
    pass  # Replace with your implementation
    
    return {
        "score": min(max(score, 300), 850),
        "rating": get_rating(score),
        "factors": factors
    }

def get_rating(score: int) -> str:
    """Convert numeric score to rating."""
    if score >= 750:
        return "Excellent"
    elif score >= 700:
        return "Good"
    elif score >= 650:
        return "Fair"
    elif score >= 600:
        return "Poor"
    else:
        return "Very Poor"

# Test your implementation
# balances = aggregator.get_total_balance("aggregator-user-001")
# tx_data = aggregator.get_all_transactions("aggregator-user-001", days=90)
# score_result = calculate_credit_score(balances, tx_data['transactions'])
# print(f"Credit Score: {score_result['score']} ({score_result['rating']})")
# for factor in score_result['factors']:
#     print(f"  - {factor}")

## Summary

In this notebook, you learned the fundamentals of Open Banking:

### Key Concepts Covered

1. **Open Banking Regulations**:
   - PSD2/PSD3 in Europe mandates banks to open APIs
   - Three types of services: AISP, PISP, CISP
   - Regulatory oversight and TPP registration

2. **Account Information Service**:
   - GET /accounts - List user accounts
   - GET /accounts/{id}/balances - Account balances
   - GET /accounts/{id}/transactions - Transaction history

3. **Payment Initiation Service**:
   - POST /payments - Initiate new payments
   - Strong Customer Authentication (SCA) requirement
   - Payment status tracking

4. **OAuth 2.0 Authorization**:
   - Authorization Code flow for web apps
   - Access tokens and refresh tokens
   - Consent management

5. **Account Aggregation**:
   - Connecting multiple banks
   - Unified view of finances
   - Cross-bank analytics

### Real-World Applications

- **Personal Finance Apps**: Mint, YNAB, Emma
- **Payment Platforms**: Transferwise, Klarna
- **Lending Services**: Credit Karma, Plaid
- **Accounting Software**: QuickBooks, Xero
- **Investment Platforms**: Robinhood, Wealthfront

### Security Considerations

- Always use HTTPS and proper authentication
- Implement rate limiting and fraud detection
- Follow data minimization principles
- Regular security audits and compliance checks
- Proper consent management and user control

### Next Steps

- Explore real Open Banking APIs (Plaid, TrueLayer, Tink)
- Study PSD2/PSD3 regulations in detail
- Learn about API security standards (FAPI, OpenID Connect)
- Understand real-world implementation challenges
- Build your own Open Banking application

### Further Reading

- [European Banking Authority - PSD2](https://www.eba.europa.eu/regulation-and-policy/payment-services-and-electronic-money)
- [Open Banking UK Standards](https://standards.openbanking.org.uk/)
- [Berlin Group NextGenPSD2](https://www.berlin-group.org/nextgenpsd2-framework)
- [Plaid Documentation](https://plaid.com/docs/)
- [TrueLayer API Reference](https://docs.truelayer.com/)